# 09 — Fairness y Contrafactuales

## 1 Introduccion

**Objetivo:** Evaluar si el modelo de primas (`stacking_ensemble`) presenta diferencias
de desempeño o tratamiento entre grupos sensibles (por ejemplo, género, edad, estado civil),
y generar explicaciones contrafactuales que respeten atributos protegidos.

Usaremos:

- [`fairlearn`](https://fairlearn.org/) para métricas desagregadas por grupo mediante `MetricFrame`.
- [`DiCE`](https://github.com/interpretml/DiCE) para generar contraejemplos (“what-if”) que reduzcan la prima.  

## 2 Carga de datos

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import pandas as pd

sys.path.append("..")

from src.config import ARTIFACTS_DIR, FEAST_TRAINSET, TARGET
from src.logging_utils import setup_logger

logger = setup_logger("fairness")

# Rutas (desde notebooks/)
OOF_PATH = Path(ARTIFACTS_DIR) / "oof_predictions.csv"
FEATURES_PATH = Path("..") / FEAST_TRAINSET

logger.info(f"Cargando OOF desde: {OOF_PATH}")
oof_df = pd.read_csv(OOF_PATH)

logger.info(f"Cargando features desde: {FEATURES_PATH}")
features_df = pd.read_parquet(FEATURES_PATH)

logger.info(f"Shapes -> oof: {oof_df.shape}, features: {features_df.shape}")

2025-11-16 17:50:45.702 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/fernando/Documentos/insurance-mlops


2025-11-16 17:50:45.713 | INFO     | __main__:<module>:20 - Cargando OOF desde: /home/fernando/Documentos/insurance-mlops/artifacts/oof_predictions.csv
2025-11-16 17:50:45 | INFO | Cargando OOF desde: /home/fernando/Documentos/insurance-mlops/artifacts/oof_predictions.csv
2025-11-16 17:50:46.177 | INFO     | __main__:<module>:23 - Cargando features desde: ../data/feast/training_set.parquet
2025-11-16 17:50:46 | INFO | Cargando features desde: ../data/feast/training_set.parquet
2025-11-16 17:50:46.270 | INFO     | __main__:<module>:26 - Shapes -> oof: (1200000, 5), features: (1200000, 39)
2025-11-16 17:50:46 | INFO | Shapes -> oof: (1200000, 5), features: (1200000, 39)


In [2]:
# DataFrame para fairness
df = features_df.copy()

df["y_true"] = oof_df[TARGET]
df["y_pred"] = oof_df["stacking_oof"]

# Errores que usaremos como métricas
df["abs_error"] = (df["y_pred"] - df["y_true"]).abs()
df["rel_error"] = (df["y_pred"] - df["y_true"]) / df["y_true"].replace(0, np.nan)

df[["y_true","y_pred","abs_error","rel_error"]].describe()

,y_true,y_pred,abs_error,rel_error
count,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06
mean,1.102545e+03,1.102542e+03,6.279259e+02,2.737453e+00
std,8.649989e+02,2.243840e+02,5.509783e+02,9.219711e+00
min,2.000000e+01,9.785008e+01,2.487872e-03,-9.688368e-01
25%,5.140000e+02,1.050782e+03,2.421203e+02,-2.603091e-01
50%,8.720000e+02,1.096837e+03,4.883546e+02,2.188489e-01
75%,1.509000e+03,1.161144e+03,8.703955e+02,1.010103e+00
max,4.999000e+03,2.257445e+03,4.708715e+03,9.008953e+01


## 3 Definir atributos sensibles y métricas

**Género**: columna Gender_Male (1 = “Male”, 0 = “Female/Other”).

**Estado civil**: Marital Status_Married (1 = casado, 0 = otros).

**Edad**: bines de Age.

In [3]:
sensitive_features = {}

# Género
if "Gender_Male" in df.columns:
    sensitive_features["gender_group"] = np.where(df["Gender_Male"] == 1, "Male", "Not_Male")

# Estado civil
if "Marital Status_Married" in df.columns:
    sensitive_features["marital_group"] = np.where(df["Marital Status_Married"] == 1, "Married", "Not_Married")

# Grupos de edad
if "Age" in df.columns:
    sensitive_features["age_group"] = pd.cut(
        df["Age"],
        bins=[17, 30, 45, 60, 90],
        labels=["18-30", "30-45", "45-60", "60+"]
    )

sensitive_features.keys()

dict_keys(['gender_group', 'marital_group', 'age_group'])

## 4 Error por grupo con Fairlearn (MetricFrame)

Fairlearn propone MetricFrame como la clase central para evaluar métricas desagregadas por grupo.

In [4]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import mean_absolute_error, mean_squared_error

from src.models import rmsle

In [5]:
def group_error_report(df, sensitive_series, group_name: str):
    """Calcula métricas de desempeño por grupo usando MetricFrame."""
    # Asegura que sea Series alineada
    sens = pd.Series(sensitive_series, index=df.index, name=group_name)
    
    mf_mae = MetricFrame(
        metrics=mean_absolute_error,
        y_true=df["y_true"],
        y_pred=df["y_pred"],
        sensitive_features=sens
    )
    
    def rmsle_simple(y_true, y_pred):
        return rmsle(y_true, y_pred)
    
    mf_rmsle = MetricFrame(
        metrics=rmsle_simple,
        y_true=df["y_true"],
        y_pred=df["y_pred"],
        sensitive_features=sens
    )
    
    rep = pd.DataFrame({
        "MAE": mf_mae.by_group,
        "RMSLE": mf_rmsle.by_group
    })
    rep["MAE_gap_max-min"] = rep["MAE"].max() - rep["MAE"].min()
    rep["RMSLE_gap_max-min"] = rep["RMSLE"].max() - rep["RMSLE"].min()
    
    print(f"\n=== Desempeño por grupo: {group_name} ===")
    display(rep)
    print("MAE global:", mf_mae.overall)
    print("RMSLE global:", mf_rmsle.overall)
    
    return rep

In [6]:
if "gender_group" in sensitive_features:
    gender_rep = group_error_report(df, sensitive_features["gender_group"], "Gender")

if "marital_group" in sensitive_features:
    marital_rep = group_error_report(df, sensitive_features["marital_group"], "Marital Status")

if "age_group" in sensitive_features:
    age_rep = group_error_report(df, sensitive_features["age_group"], "Age Group")


=== Desempeño por grupo: Gender ===


,MAE,RMSLE,MAE_gap_max-min,RMSLE_gap_max-min
Gender,,,,
Male,628.307323,1.130678,0.766105,0.003592
Not_Male,627.541218,1.127086,0.766105,0.003592


MAE global: 627.9259119551666
RMSLE global: 1.1288911108945763

=== Desempeño por grupo: Marital Status ===


,MAE,RMSLE,MAE_gap_max-min,RMSLE_gap_max-min
Marital Status,,,,
Married,625.831275,1.127160,3.11979,0.002577
Not_Married,628.951064,1.129737,3.11979,0.002577


MAE global: 627.9259119551666
RMSLE global: 1.1288911108945763

=== Desempeño por grupo: Age Group ===


,MAE,RMSLE,MAE_gap_max-min,RMSLE_gap_max-min
Age Group,,,,
18-30,629.708333,1.129531,2.97064,0.008074
30-45,627.862103,1.128161,2.97064,0.008074
45-60,626.737693,1.127351,2.97064,0.008074
60+,627.019138,1.135425,2.97064,0.008074


MAE global: 627.9259119551666
RMSLE global: 1.1288911108945763


### Evaluación de equidad (Fairlearn)

Aplicamos `MetricFrame` de Fairlearn para desagregar el error de regresión (MAE y RMSLE) por género, estado civil y grupo de edad. Esta estrategia es consistente con las recomendaciones de Fairlearn de analizar métricas por subgrupos sensibles y comparar sus diferencias en lugar de mirar solo el error global.   

En nuestro caso:

- **Género**  
  - MAE: 628.3 para `Male` vs 627.5 para `Not_Male`.  
  - RMSLE: 1.1307 vs 1.1271.  
  - El gap max–min en MAE es ≈0.12 % del MAE global y en RMSLE es ≈0.003 puntos → diferencias muy pequeñas.

- **Estado civil**  
  - MAE: 625.8 para `Married` vs 629.0 para `Not_Married`.  
  - RMSLE: 1.1272 vs 1.1297.  
  - El gap max–min ronda el 0.50 % del MAE global, lo que indica solo un **ligero** aumento de error en el grupo `Not_Married`.

- **Grupo de edad**  
  - Los MAE por grupo están todos muy cercanos al valor global (entre 626.7 y 629.7).  
  - El grupo `60+` presenta un RMSLE algo mayor (1.1354), pero la diferencia absoluta frente al resto sigue siendo pequeña (gap ≈0.008).

**Conclusión:**  
El modelo de primas muestra **desempeño muy parecido entre los diferentes grupos** de género, estado civil y edad. Los gaps max–min en MAE y RMSLE son menores al 1 % respecto al error global, por lo que **no se observa un sesgo fuerte en términos de error de predicción entre estos grupos**, aunque sí hay indicios de un error algo mayor en personas `Not_Married` y en el grupo de edad `60+`, que conviene seguir monitoreando en futuras iteraciones del modelo.


## 5 “Quién recibe primas altas” por grupo

Aquí adaptamos la idea de demographic parity: en clasificación se mide si la tasa de resultado positivo es similar entre grupos.

En este caso, definimos “resultado negativo” como recibir una prima alta.

Vamoa a crear una etiqueta binaria de “prima alta”

alto = por encima del percentil 75 de la prima predicha:

In [7]:
q_high = df["y_pred"].quantile(0.75)
df["is_high_premium"] = (df["y_pred"] > q_high).astype(int)

q_high

np.float64(1161.1435042065718)

In [8]:
from fairlearn.metrics import selection_rate

def high_premium_rate(y_true, y_pred):
    # y_pred es la etiqueta binaria is_high_premium
    return selection_rate(y_true=y_pred, y_pred=y_pred)

def high_premium_by_group(df, sensitive_series, group_name):
    sens = pd.Series(sensitive_series, index=df.index, name=group_name)
    mf = MetricFrame(
        metrics=high_premium_rate,
        y_true=df["is_high_premium"],  # no se usa realmente
        y_pred=df["is_high_premium"],
        sensitive_features=sens
    )
    rep = mf.by_group.rename("HighPremiumRate").to_frame()
    rep["gap_max-min"] = rep["HighPremiumRate"].max() - rep["HighPremiumRate"].min()
    print(f"\n=== Tasa de primas altas por grupo: {group_name} ===")
    display(rep)
    print("Tasa global:", mf.overall)
    return rep

In [9]:
if "gender_group" in sensitive_features:
    high_gender = high_premium_by_group(df, sensitive_features["gender_group"], "Gender")

if "age_group" in sensitive_features:
    high_age = high_premium_by_group(df, sensitive_features["age_group"], "Age Group")


=== Tasa de primas altas por grupo: Gender ===


,HighPremiumRate,gap_max-min
Gender,,
Male,0.249924,0.000153
Not_Male,0.250077,0.000153


Tasa global: 0.25

=== Tasa de primas altas por grupo: Age Group ===


,HighPremiumRate,gap_max-min
Age Group,,
18-30,0.256853,0.013983
30-45,0.251873,0.013983
45-60,0.242869,0.013983
60+,0.247772,0.013983


Tasa global: 0.25


### ¿Quién recibe primas altas? (paridad demográfica aproximada)

Definimos “prima alta” como estar por encima del percentil 75 de la prima predicha (≈ 1161). A partir de esta etiqueta binaria (`is_high_premium`), calculamos la **tasa de primas altas** por grupo usando `selection_rate`, siguiendo la idea de *demographic parity* de Fairlearn (comparar la proporción de resultados “positivos” entre grupos sensibles).   

- **Género**  
  - Hombres: 0.2499  
  - No-hombres: 0.2501  
  - Tasa global: 0.25  
  → Las tasas son prácticamente idénticas; **no se observa desbalance** en la probabilidad de recibir una prima alta por género.

- **Grupo de edad**  
  - 18–30: 0.2569  
  - 30–45: 0.2519  
  - 45–60: 0.2429  
  - 60+: 0.2478  
  - Tasa global: 0.25  
  → Las diferencias entre grupos de edad son pequeñas (gap ≈ 0.014). El grupo 18–30 recibe ligeramente más primas altas y 45–60 ligeramente menos, pero las variaciones son moderadas.

**Conclusión:**  
Con este umbral (percentil 75), el modelo asigna primas altas con tasas muy similares entre géneros y con diferencias moderadas entre grupos de edad, por lo que **no se aprecia un sesgo fuerte en quién termina recibiendo primas altas**. Estas tasas deberían seguir monitorizándose si se cambia el umbral o se reentrena el modelo.

## 6 Contrafactuales con DiCE

La documentación de DiCE muestra cómo generar contraejemplos tanto para clasificadores como para regresores, definiendo un rango deseado para la salida (“desired_range”).

La idea que tenemos es:

“Dado este cliente, ¿qué cambios mínimos (en ingresos, hábitos, etc.) llevarían a que su prima bajara al menos un X%? Sin permitir cambiar género ni edad.”

In [10]:
import dice_ml
from dice_ml.utils import helpers

from src.data import split_xy

# Volvemos a construir X, y desde el training set de Feast
X_full, y_full = split_xy(features_df)  # ojo: aquí features_df es el training_set.parquet

df_dice = X_full.copy()
df_dice[TARGET] = y_full

df_dice.head()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,psd_year,psd_month,...,Location_Urban,Location_Unknown,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Exercise Frequency_Unknown,Property Type_Condo,Property Type_House,Property Type_Unknown,Premium Amount
0,19.0,498.0,2.0,27.068329,0.0,17.0,480.0,5.0,2019.0,8.0,...,False,False,False,True,False,False,True,False,False,1328.0
1,45.0,102043.0,0.0,36.477553,1.0,14.0,543.0,6.0,2019.0,8.0,...,False,False,False,False,False,False,True,False,False,20.0
2,33.0,7894.0,1.0,35.986064,0.0,6.0,445.0,5.0,2019.0,8.0,...,False,False,False,True,False,False,True,False,False,730.0
3,58.0,47253.0,2.0,24.981910,1.0,14.0,485.0,8.0,2019.0,8.0,...,True,False,False,True,False,False,True,False,False,2979.0
4,40.0,3050.0,3.0,18.849237,1.0,19.0,716.0,7.0,2019.0,8.0,...,False,False,False,False,False,False,False,False,False,688.0


In [11]:
continuous_features = [
    c for c in df_dice.columns
    if c != TARGET                                  # excluir target
    and df_dice[c].dtype in ("int16","int32","int64","float32","float64")
    and not c.startswith("Gender_")
]

In [12]:
continuous_features

['Age',
 'Annual Income',
 'Number of Dependents',
 'Health Score',
 'Previous Claims',
 'Vehicle Age',
 'Credit Score',
 'Insurance Duration',
 'psd_year',
 'psd_month',
 'psd_dow',
 'psd_month_sin',
 'psd_month_cos',
 'Policy Type',
 'Education Level',
 'Customer Feedback']

In [13]:
data_dice = dice_ml.Data(
    dataframe=df_dice,
    continuous_features=continuous_features,
    outcome_name=TARGET
)

# Usamos tu ensemble como modelo sklearn
import src.train as train
StackingEnsemble = train.StackingEnsemble

import joblib
from src.config import MODELS_DIR

model_path = MODELS_DIR / "stacking_ensemble.joblib"
ensemble_model = joblib.load(model_path)

# ⚠️ Wrapper para usar el StackingEnsemble con DiCE
import pandas as pd
import numpy as np

class SklearnDiceEnsemble:
    def __init__(self, model, feature_names):
        self.model = model
        # columnas que el modelo espera (todas las features, sin el TARGET)
        self.feature_names = list(feature_names)

    def predict(self, X):
        # Aseguramos un DataFrame con los nombres correctos
        if isinstance(X, np.ndarray):
            X_df = pd.DataFrame(X, columns=self.feature_names)
        else:
            X_df = X.copy()
            if not isinstance(X_df, pd.DataFrame):
                X_df = pd.DataFrame(X_df, columns=self.feature_names)
            else:
                # reordenar y asegurar que estén todas las columnas
                X_df = X_df.reindex(columns=self.feature_names)

        # Convertir columnas tipo object a numéricas si hace falta
        for col in X_df.columns:
            if X_df[col].dtype == "object":
                X_df[col] = pd.to_numeric(X_df[col], errors="coerce")

        # Llamar al ensemble original
        return self.model.predict(X_df)

# Usamos el wrapper en lugar de pasar el ensemble directo
dice_wrapper = SklearnDiceEnsemble(
    model=ensemble_model,
    feature_names=df_dice.drop(columns=[TARGET]).columns
)

model_dice = dice_ml.Model(
    model=dice_wrapper,
    backend="sklearn",
    model_type="regressor"
)

In [14]:
exp = dice_ml.Dice(
    data_interface=data_dice,
    model_interface=model_dice,
    method="random"  # o "genetic" si quieres algo más sofisticado
)

In [15]:
# Elegir un caso “difícil” (error alto) como ejemplo
worst_idx = df["abs_error"].idxmax()
logger.info(f"Ejemplo con mayor error: idx={worst_idx}, error={df.loc[worst_idx,'abs_error']}")

query_instance = df_dice.loc[[worst_idx]].drop(columns=[TARGET])
query_instance

2025-11-16 17:51:28.782 | INFO     | __main__:<module>:3 - Ejemplo con mayor error: idx=100201, error=4708.715413307085
2025-11-16 17:51:28 | INFO | Ejemplo con mayor error: idx=100201, error=4708.715413307085


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,psd_year,psd_month,...,Location_Suburban,Location_Urban,Location_Unknown,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Exercise Frequency_Unknown,Property Type_Condo,Property Type_House,Property Type_Unknown
100201,60.0,76684.0,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,True,False,False,False,False,False,False,True,False,False


In [16]:
# Definir rango deseado para la prima
current_pred = df.loc[worst_idx, "y_pred"]
desired_max = current_pred * 0.8  # 20% menos
desired_range = [0, desired_max]
desired_range

[0, np.float64(216.2276693543315)]

In [17]:
# columnas que NO queremos que cambien
immutable = []
if "Gender_Male" in df_dice.columns:
    immutable.append("Gender_Male")
if "Age" in df_dice.columns:
    immutable.append("Age")

# solo se podrán modificar las columnas que NO estén en immutable
features_to_vary = [c for c in X_full.columns if c not in immutable]

In [18]:
# Congelar atributos protegidos
permitted = {}
if "Age" in df_dice.columns:
    age_val = float(df_dice.loc[worst_idx, "Age"])
    permitted["Age"] = [age_val, age_val]
permitted

{'Age': [60.0, 60.0]}

In [19]:
dice_exp = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_range=desired_range,
    features_to_vary=features_to_vary,      # luego restringimos con permitted_range
    permitted_range=permitted or None
)

cf_df = dice_exp.cf_examples_list[0].final_cfs_df
cf_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.65s/it]


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,psd_year,psd_month,...,Location_Urban,Location_Unknown,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Exercise Frequency_Unknown,Property Type_Condo,Property Type_House,Property Type_Unknown,Premium Amount
0,60.0,52038.9,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,202.0
1,60.0,52854.0,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,201.0
2,60.0,48361.0,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,195.0


In [20]:
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 321.0)


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,psd_year,psd_month,...,Location_Urban,Location_Unknown,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Exercise Frequency_Unknown,Property Type_Condo,Property Type_House,Property Type_Unknown,Premium Amount
0,60.0,76684.0,0.0,36.508663,0.0,11.0,709.0,7.0,2020.0,1.0,...,True,True,True,True,True,True,True,True,True,321.0



Diverse Counterfactual set (new outcome: [0, np.float64(216.2276693543315)])


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,psd_year,psd_month,...,Location_Urban,Location_Unknown,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Exercise Frequency_Unknown,Property Type_Condo,Property Type_House,Property Type_Unknown,Premium Amount
0,60.0,52038.9,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,201.585709
1,60.0,52854.0,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,200.862930
2,60.0,48361.0,0.0,36.508664,0.0,11.0,709.0,7.0,2020.0,1.0,...,False,False,False,False,False,False,True,False,False,195.206512


### Contrafactuales con DiCE

Para el cliente elegido (60 años, prima actual ≈ 321):

- Mantuvimos **edad y género fijos** y pedimos a DiCE escenarios donde la prima bajara al menos un **20%**.
- Los contrafactuales encontrados reducen la prima a ≈ **202–208**, es decir, una caída de ~35% respecto a la prima original.
- En estos escenarios, la edad permanece en 60 años y el tipo de propiedad sigue siendo *Condominio*, pero cambian otras variables:
  - Se reduce el **Annual Income** (de ~76k a ~50k).
  - Se modifican algunas dummies de **ubicación** (de `Location_Urban = True` a `False`).
  - Se ajustan variables de **frecuencia de ejercicio** y otros indicadores de estilo de vida (se pasan de valores “True/Unknown” a combinaciones distintas).

Interpretación:

- Estos contrafactuales muestran que, según el modelo, **cambios moderados en ingresos, ubicación y hábitos declarados** serían suficientes para bajar significativamente la prima, incluso manteniendo constantes atributos no editables como edad y género.
- No deben leerse como recomendaciones literales (no tiene sentido “bajar ingresos” a propósito), sino como una forma de identificar **qué variables está usando el modelo como palancas fuertes** para ajustar el riesgo en este tipo de perfil.